<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/FineTuning_phi_2_with_PRFT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Checkout my [Twitter(@rohanpaul_ai)](https://twitter.com/rohanpaul_ai) for daily LLM bits

##  Fine Tuning Phi 1_5 with PEFT and QLoRA

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)

[![Imgur](https://imgur.com/ensYOlt.png)](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)



In [ ]:
#安装需要依赖
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7
!pip install -q huggingface_hub sentencepiece einops nvidia-ml-py3

In [2]:
# import 密钥，token
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

GPU memory occupied: 258 MB.


In [4]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [8]:
model

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

## A little hack to make `get_peft_model()` method faster by avoiding `kaiming_uniform` initialization

In [9]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init  # Importing the init module from PyTorch
import bitsandbytes as bnb
from contextlib import contextmanager

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

@contextmanager
def no_kaiming():
    old_iku = init.kaiming_uniform_
    init.kaiming_uniform_ = noop
    try: yield
    finally: init.kaiming_uniform_ = old_iku

_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias=True,
               device=None, **kwargs):
    with no_kaiming():
        return _old_4init(self, input_features, output_features, bias=bias,
                          device=device, **kwargs)



def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
              memory_efficient_backward=False, threshold=0.0, index=None, device=None):
    with no_kaiming():
        return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
                          memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [10]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 1,529,256,960 || trainable%: 0.5142575908237161


![](assets/lora-4.png)

In [12]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (transformer): PhiModel(
        (embd): Embedding(
          (wte): Embedding(51200, 2560)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (h): ModuleList(
          (0-31): 32 x ParallelBlock(
            (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (resid_dropout): Dropout(p=0.1, inplace=False)
            (mixer): MHA(
              (rotary_emb): RotaryEmbedding()
              (Wqkv): Linear4bit(
                in_features=2560, out_features=7680, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=7680, bias=False)
                )
  

In [13]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return tokenized_text

In [15]:
data = load_dataset("BI55/MedText", "default", split="train")

data_df = data.to_pandas()

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: "Prompt: " + x["Prompt"] + " Completion: " + x["Completion"], axis=1)

In [17]:
data = Dataset.from_pandas(data_df)

tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/1412 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1412
})

In [18]:
training_arguments = TrainingArguments(
        output_dir="microsoft/phi-2-finetuned-med-text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1
    )

In [19]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

# trainer.push_to_hub()

Step,Training Loss
100,1.310400
200,1.226500
300,1.210800
400,1.177700
500,1.151400
600,1.157600
700,1.144000
800,1.116600
900,1.108600
1000,1.094800


TrainOutput(global_step=1000, training_loss=1.169828224182129, metrics={'train_runtime': 1082.1001, 'train_samples_per_second': 3.697, 'train_steps_per_second': 0.924, 'total_flos': 1.4845451372544e+16, 'train_loss': 1.169828224182129, 'epoch': 2.83})

## Saving

In [20]:
model.save_pretrained("microsoft/phi-2-finetuned-med-text")

In [21]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", trust_remote_code=True, torch_dtype=torch.float32)

peft_model = PeftModel.from_pretrained(model, "microsoft/phi-2-finetuned-med-text", from_transformers=True)

model = peft_model.merge_and_unload()

# model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
model.save_pretrained("microsoft/phi-2-finetuned-med-text")

# model.push_to_hub("llm-exp/phi-1_5-finetuned-med-text")

## The overall flow

1. load the base model

2. train the base model

3. save the LoRA adapter

4. reload the base model at full precision

5. merge the LoRA weights with the base model

6. save


# Inference

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("phi-1_5-finetuned-med-text", trust_remote_code=True, torch_dtype=torch.float32)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('I am having a headache with but no fever - what could be the cause', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=512)

text = tokenizer.batch_decode(outputs)[0]

print(text)
